In [1]:
_RIOT_API_KEY='RGAPI-ea8304f2-1e97-49e1-9ad2-1ba6e618b489'
from riotwatcher import LolWatcher
import helper_functions as help
lol_watcher = LolWatcher(_RIOT_API_KEY)



1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

In [ ]:
match_list=help.get_matchlist_by_summoner_name('souporsecret', lolwatcher_connection)
len(x)